# Finding the "best" model

1. The Curse of Dimensionality
2. Comparing models
3. Subset selection
4. Stepwise selection (Forward/Back)

The lecture draws from Chapter 6 of James, G., Witten, D., Hastie, T., & Tibshirani, R. (2013). "An introduction to statistical learning: with applications in r."

---
# 1. The Curse of Dimensionality

When trying to find the best $f()$ to explain $Y=f(X)$, the goal is to explain as much variability in $Y$ as possible. So it seems intuitive to make your model as complex as possible: i.e., include as many variables in $X$ as possible so as to explain every part of $Y$. 

However, when you are not working with sufficiently large sampled data sets (i.e., when $n$ isn't really large) with respect to the number of variables in $X$ (i.e., $p$), then you are said to have a model with _high dimensionality_.

High dimensional models are very problematic in two ways. 

* **Prediction accuracy:** If $n$ is not large relative to $p$, then methods like least squares fits will have high variance, resulting in overfitting, which impairs test accuracy for predicting new data. In the case where p > n _there is no longer a unique least squares coefficient estimate_, which means that you can never find the best $f()$.

* **Model interpretability:** Having a lot of variables in $X$ increases complex interrelationships between your predictor variables. As we saw in the explanation on building models with interaction terms, this impairs your ability to interpret specific $X \rightarrow Y$ relationships. Thus you may want to remove irrelevant variables to reduce the complexity of your model since many features may not predict the response variable. This is a process known as _feature selection_. 

Thus when evaluating different models, there is a push for _parsimony_ such that you want to construct the simplest model that explains the most variance in the relationship between $X$ and $Y$.



---
# 2. Comparing models

**The Problem:** Training fits for any model will increase naturally with $p$. In order to meet the principle of parsimony, you'll need to need model evaluation methods that account for model complexity (i.e., differences in $p$) as well as residual error. 


There are two general approaches to this problem:

* **Validation:** Directly estimate test error using a cross-validation approach.

* **Bias adjustment:** Adjust estimate of residual error by accounting for model complexity (i.e. by accounting for bias).

Let's consider each of these in turn.

<br>

## Validation approaches

Using cross validation approaches discussed in the cross-validation lecture (Chapter 5), we can directly test model prediction error. Since model complexity only impacts training error (or fitting error) and noise is independent across samples, the number of predictors shouldn’t really impact test error.

![Validation approaches](imgs/L13_kNNVarianceBias.png)

<br>

## Bias adjustment

Sometimes it's not possible to do a  validation test. For example, some non-parametric approaches don't lend themselves to a validation approach. So the alternative approach is to take model complexity into account in your residual error estimate. Let's go over three of these methods.

* **Cp (Mallow's Cp):** If you take a model with $p$ predictors (in the book they use the term _d_ to indicate model complexity, but I'll use the term we've used consistently so far), then you penalise the residual square error (RSS) by inflating the residual error by $p$. 
    * $C_p = \frac{1}{n}(RSS + 2p\hat{\sigma}^2) $ 
    * $\hat{\sigma}^2$ , estimate of variance associated with each response in the model.
    * Thus $C_p$ penalizes the model fit by $2p\hat{\sigma}^2$
    
* **AIC (Akaike Information Criterion):** For all models that are fit by maximizing a likelihood function, you can modify $C_p$ slightly such that:
    * $AIC = \frac{1}{n\hat{\sigma}^2}(RSS + 2p\hat{\sigma}^2)$
    * You can rewrite AIC as $AIC = 2p - 2ln(L)$
        * L is the maximized likelihood function of the model $M$ (i.e., $L=p(x|\theta, M)$, where $\theta$ are the fit parameters of the model.
        * You'll often see the above equation with $p$ replaced by $k$.

* **BIC (Bayesian Information Criterion):** You can reframe AIC as a baysian problem instead. 
    * $BIC = \frac{1}{n}(RSS + log(n)p\hat{\sigma}^2)$ 
    * Alternatively: $BIC = -2ln(L) + p ln(n)$
    * Here _n_ is still the number of observations in the model. 

<br>

Notice that Cp, AIC, & BIC all rely on the redidual sums of squares in their calculation. This means that they are all a measure of the amount of information **lost** by your model. Thus the model that returns the _lowest_ value is the model that is the most parsimonious.

<br>

However, let's consider an alternative, bias adjusted method.

* **Adjusted $r^2$:** Rather than penalize the RSS, penalize the $r^2$ calculation of the mode.
    * Adjusted $r^2 = 1 - \frac{\frac{RSS}{n-d-1}}{\frac{TSS}{n-1}} $
    * TSS is the total sums of squares (i.e., $\sum_{i=1}{n} (y_i - \bar{y})^2$
    * This is interpreted as any other $r^2$, bigger is better.
    
<br>
<br>
Now let's compare our three methods. The figure below shows a model comparison from the textbook on a regression model using the _Credit_ data set, where models with increasing complexity (i.e. flexibilty) are evaluated using either Cp, BIC, or Adjusted $r^2$. The "X" highlights the model that would be selected according to each criterion.

![Cp, BIC, Adjusted r-square](imgs/L16_AdjustedMethods.png)
 
Notice that each method identifies a slightly different model. Cp picks a model with 6 parameters, BIC picks a model with 4 parameters, and the adjusted $r^2$ picks a model with 6. This highlights the fact that it is generally preferable to use multiple methods to evaluate a model and take the concensus.


---
# 3. Subset selection

Now that we have tools to evaluate models of different complexities, we can look at approaches for _choosing exactly which predictor variables to include in our model_. The most thorough approach is _best subset selection_.

* **Best Subset Selection:** Select the subset of predictors that do the best at meeting an bias adjusted criterion by trying all possible model combinations.

The simplest way to think about this is by example. Let's consider a model where _p = 3_. Now this isn't exactly a high dimensional case, but it gives you the iniuition of the best subsets selection algorithm.

Here the full model is:

$$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_3 + \epsilon $$

Now the question is what combination of $X_1$, $X_2$, and $X_3$ provides the most parsimonious model that maximizes your fits with the fewest number of parameters? With full subsets selection you try _all combinations_ of variables. Since _p=3_, this means we have to test all $3! = 3 * 2 * 1 = 6$ models plus the _null model_. 

$$ Y = \beta_0$$
$$ Y = \beta_0 + \beta_1 X_1$$
$$ Y = \beta_0 + \beta_2 X_2$$
$$ Y = \beta_0 + \beta_3 X_3$$
$$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2$$
$$ Y = \beta_0 + \beta_1 X_1 + \beta_3 X_3$$
$$ Y = \beta_0 + \beta_2 X_2 + \beta_3 X_3$$

For each model we would calculate a bias adjusted model fit measure (e.g., AIC or Cp) and select the single model that gives the best adjusted fit.

The textbook presents the full, formal algorithm for best subset approaches.

![Best subset](imgs/L16_BestSubsetsAlgo.png)

Note that this approach is computationally intensive because you have to try all possible combinations of the model (i.e., the total number of models is $p! + 1$). So the number of models that you have to test _increases exponentially_ as _p_ gets larger. Thus the total space of models you have to test is $2^p$ in order to find the right subset.

---
# 4. Stepwise selection (forward/backward)

The biggest problem with model subset selection is computational costs, since the total space of models tested is $2^p$. As _p_ gets very large this can be prohibitively expensive. An alternative to full subset evaluation is to do _stepwise_ evaluations of different models. Here we look at two stepwise algorithms.

## Forward stepwise selection:

Here you start with your null model (i.e., p=0) and add predictor variables one at a time to slowly increase model complexity. Then stop when the bias-adjusted model fit jumps according to a pre-determined amount (i.e., a threshold). 

If we return to our _p=3_ example above we would try all combinations of models with the same complexity (_k_). We start with _k=0_ (i.e. the null model) and estimate it's bias-adjusted fit ($M_0$)

$$ Y = \beta_0$$

Then move up to the next most complex models, where _k=1_.

$$ Y = \beta_0 + \beta_1 X_1$$
$$ Y = \beta_0 + \beta_2 X_2$$
$$ Y = \beta_0 + \beta_3 X_3$$

For this example, let's say that the model that included the $X_1$ term provided the best bias-adjusted fit. We will save that fit value as ($M_1$) and now only evaluate any models moving forward that include this term. When we increase _k_ to 2, we thus have only 2 version of the model to evaluate.

$$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2$$
$$ Y = \beta_0 + \beta_1 X_1 + \beta_3 X_3$$

Let's say that here the model that includes both $X_1$ and $X_3$ provided the best bias-adjusted fit. This fit gets saved as ($M_2$). We would only include those terms in any model moving forward.

Finally, the most complex model where _k=p_. Since there is only one version of this model we estimate it's model fit as $M_3$.

$$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_3 $$

In order to find the best model, we take the $M_k$ with the best bias-adjusted fit and use that model as our "best" model. In this example, let's say that $M_2$ was the best fit value out of $M_0, M_1, M_2, M_3$. In this case we would say that our best model is

$$ Y = \beta_0 + \beta_1 X_1 + \beta_3 X_3$$

The formal algorithm for this process is summarized in the text as.

![Forward Stepwise](imgs/L16_ForwardStepwise.png)

Now in our toy example above, doing forward stepwise selection only saved us from running one model in this example. However, as _p_ becomes large, this stepwise approach can dramatically reduce the number of models that need to be tested. Specifically, instead of $2^p$ models, forward stepwise selection requires only testing $1+p(p+1)/2$ models. 

<br>

## Backward stepwise selection

As you can imagine, where forward stepwise selection starts with the most simple model and works forward, backward stepwise selection starts with the most complex and works down in complexity from there.

In the example we have been using thus far with _p=3_, we would start with the most complex model (i.e., _k=3_)

$$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_3 $$

and save the bias-adjusted model fit as ($M_3$). 

From there we would reduce complexity by one, such that _k=2_.

$$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2$$
$$ Y = \beta_0 + \beta_1 X_1 + \beta_3 X_3$$
$$ Y = \beta_0 + \beta_2 X_2 + \beta_3 X_3$$

As with forward stepwise we would select the model with the best fit and set that as the representative for this class of models (i.e., $M_2$). Let's say that the model with $X_1$ and $X_3$ provided the best bias-adjusted fit here. Now, when we go down in complexity such that _k=1_ we only evaluate models with those terms.

$$ Y = \beta_0 + \beta_1 X_1$$
$$ Y = \beta_0 + \beta_3 X_3$$

Whichever term provided the best bias-adjusted fit becomes the representative for that model complexity ($M_1$).

Finally, we get the bias-adjusted fit on the null model ($M_0$).

$$ Y = \beta_0$$

The logic of the selection process is the same as with forward stepwise selection: find the $M_k$ with the best fit and take the model that gave you that fit as the "best" model.

Here is the formal algorithm for backward stepwise selection from the text.

![Backward stepwise](imgs/L16_BackwardStepwise.png)

Notice that the number of models you evaluated is the same as in the forward stepwise selection $1+p(p+1)/2$. Thus they have the same computational complexity. However, backward selection requires that _n_ is larger than _p_ at all times (so that the full model can be fit). Whereas forward stepwise seleciton can be used even when $n<p$. **Therefore forward stepwise selection is the only viable method to use when _p_ is very large.**

